In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
from simpletransformers.t5 import T5Model, T5Args

nmv = pd.read_parquet(f"transformations/NMV_hazm.parquet")
original = pd.read_parquet(f"transformations/original.parquet")
nt = ["Matthew", "Mark", "Luke", "John", "Acts", "Romans", "I Corinthians", "II Corinthians", "Galatians", "Ephesians", "Philippians", "Colossians", "I Thessalonians", "II Thessalonians", "I Timothy", "II Timothy", "Titus", "Philemon", "Hebrews", "James", "I Peter", "II Peter", "I John", "II John", "III John", "Jude", "Revelation of John"]
nmv_ot = nmv.loc[~nmv.book.isin(nt)].copy()
nmv_nt = nmv.loc[nmv.book.isin(nt)].copy()
original_ot = original.loc[~original.book.isin(nt)].copy()
original_nt = original.loc[original.book.isin(nt)].copy()
def prepare_translation_datasets(nmv_list, original_list):
    def ProcessDF(src, trg):
        data = []
        for original, fa in zip(trg, src):
            data.append(["translate original to fa", original, fa])
            data.append(["translate fa to original", fa, original])

        return pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])
        
    fa_text = (
        nmv_list
        .groupby(["book", "idx_chapter", "idx_verse"])
        ["word"]
        .agg(" ".join)
        .to_list()
    )

    original_text = (
        original_list
        .groupby(["book", "idx_chapter", "idx_verse"])
        ["eng_word"]
        .agg(" ".join)
        .apply(lambda x:x.replace("/",""))
        .to_list()
    )

    fa_train, fa_test, original_train, original_test = train_test_split(fa_text, original_text, test_size = 0.2)

    train_df = ProcessDF(fa_train, original_train)

    eval_df = ProcessDF(fa_test, original_test)

    return train_df, eval_df
ot_train_df, ot_eval_df = prepare_translation_datasets(nmv_ot, original_ot)

ot_train_df.to_csv("transformations/ot_train.tsv", sep="\t")
ot_eval_df.to_csv("transformations/ot_eval.tsv", sep="\t")
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
train_df = pd.read_csv("transformations/ot_train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("transformations/ot_eval.tsv", sep="\t").astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 20
model_args.eval_batch_size = 20
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Heb-Fa OT Translation"

model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=False)

model.train_model(train_df, eval_data=eval_df)


ImportError: cannot import name '_sparse' from 'torch._C' (c:\Users\saaam\farsi-strongs\st38\lib\site-packages\torch\_C.cp38-win_amd64.pyd)